In [1]:
!pip install accelerate datasets bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

In [2]:
from transformers import BitsAndBytesConfig
import transformers
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
import pandas as pd

In [3]:
dataset = load_dataset("persiannlp/parsinlu_query_paraphrasing")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1830
    })
    test: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1916
    })
    validation: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 898
    })
})

In [ ]:
# test_data = dataset['test']
# test_data[15]

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# model_name = "ViraIntelligentDataMining/PersianLLaMA-13B"

# quantization_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4")

# pipe = pipeline(
#     "text-generation",
#     model = model_name,
#     device_map="auto",
#     model_kwargs={    "low_cpu_mem_usage":True,
#     "quantization_config":quantization_config}
# )

pipe = transformers.pipeline(
    "text-generation",
    model="NousResearch/Meta-Llama-3-8B-Instruct",
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
    device_map="auto",
)
pipe

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["test"])
test_df

,q1,q2,category,label
0,آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنتر...,چه چیزی روح فرد را می شکند؟,qqp,0
1,چگونه می توانم تماشای فیلم های پورنو را متوقف ...,برای جلوگیری از تماشای کامل پورنو باید چه کاری...,qqp,1
2,چه کسانی امام علی را خدا میدانند؟,چه کسانی می توانند امام زمان را ببینند؟,natural,0
3,آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایال...,شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا...,qqp,1
4,چگونه می توانم سوالی را در این باره بپرسم؟,چگونه می توانم سوال بپرسم؟,qqp,0
...,...,...,...,...
1911,چه چیزهایی روزه زن را باطل میکند اهل سنت؟,چه چیزهایی روزه زن را باطل میکند مکارم شیرازی؟,natural,0
1912,وای چه پسری وای عجب تاج سری؟,دانلود اهنگ پسر وای وای چه پسری وای عجب تاج سری؟,natural,1
1913,از چه زمانی حرکات جنین کم میشود؟,از چه زمانی حرکات جنین حس میشود؟,natural,0
1914,کدام یک از شرکتها دارای رشد شغلی بهتری دارند؟ ...,۴ لوله ورودی و ۲ لوله خروجی ، ورودی آب را پیدا...,qqp,0


In [7]:
pipe

In [11]:
## https://huggingface.co/NousResearch/Meta-Llama-3-8B-Instruct
# messages = [
#     {"role": "system", "content": "Answer questions"}, # chat template
#     {"role": "user", "content": "Who are you?"},
# ]

# prompt = pipe.tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
# )
# ## prompt
# terminators = [
#     pipe.tokenizer.eos_token_id,
#     pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# ## terminators

# outputs = pipe(
#     prompt,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
#     pad_token_id=pipe.tokenizer.eos_token_id
# )
# ##outputs

# print(outputs[0]["generated_text"][len(prompt):])

I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm here to help answer questions, provide information, and even have a chat! What's on your mind?


In [9]:
# predictions = []
# labels = []
# examples = []

# for example in test_df.values[:5]:
#   # print(example[0])
#   question1 = example[0]
#   question2 = example[1]
#   label = example[2]

#   input_text = f"آیا این دو سوال مترادف هستند?\n سوال 1: {question1}\n سوال 2: {question2}"
#   result = pipe(input_text,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
#     pad_token_id=pipe.tokenizer.eos_token_id)

#   # predictions.append(predicted_label)
#   # labels.append(int(label))
#   # examples.append((question1, question2, int(label), predicted_label))
#   print(result)

# # test_data[:5]
# # test_df.iloc[:5]

In [34]:
def zero_shot_prompt(q1, q2):
    return (
        "دو سوال داریم : \n"
        "میخواهیم مترادف بودن این دو سوال را بررسی کنیم؟ \n"
        "اگر مترادف هستند بگو بله "
        "اگر مترادف نیستند بگو خیر \n"
        f"سوال اول: {q1} \n"
        f"سوال دوم: {q2}"
    )

In [33]:
def one_shot_prompt(q1, q2, example):
    example_prompt = (
        f"سوال اول: {example['q1']} \n"
        f"سوال دوم: {example['q2']} \n"
        f"جواب: {'بله' if example['label'] == 1 else 'خیر'}"
    )
    return (
        "دو سوال داریم : \n"
        "میخواهیم مترادف بودن این دو سوال را بررسی کنیم؟ \n"
        "اگر مترادف هستند بگو بله "
        "اگر مترادف نیستند بگو خیر \n"
        f"مثال: \n{example_prompt} \n"
        f"حال با دیدن مثال جواب دهید. \n"
        f"سوال اول: {q1} \n"
        f"سوال دوم: {q2}"
    )

In [32]:
def five_shot_prompt(q1, q2, examples):
    example_prompts = "\n".join(
        f"سوال اول: {row['q1']} \nسوال دوم: {row['q2']} \nجواب: {'بله' if row['label'] == 1 else 'خیر'}"
        for _, row in examples.iterrows()
    )
    return (
        "دو سوال داریم : \n"
        "میخواهیم مترادف بودن این دو سوال را بررسی کنیم؟ \n"
        "اگر مترادف هستند بگو بله "
        "اگر مترادف نیستند بگو خیر \n"
        f"مثال‌ها: \n{example_prompts} \n"
        f"حال با دیدن مثال‌ها جواب دهید. \n"
        f"سوال اول: {q1} \n"
        f"سوال دوم: {q2}"
    )

In [18]:
def n_shot_prompt(scenario, q1, q2, examples=None):
    if scenario == "Zero-shot":
        return zero_shot_prompt(q1, q2)
    elif scenario == "One-shot":
        return one_shot_prompt(q1, q2, examples.iloc[0])
    elif scenario == "Five-shot":
        return five_shot_prompt(q1, q2, examples)

In [19]:
def generated_text(prompt, max_new_tokens=64):
    template = "Answer the questions in a single word"
    messages = [
        {"role": "system", "content": template},
        {"role": "user", "content": prompt},
    ]

    prompt = pipe.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    return outputs[0]["generated_text"][len(prompt):]

In [21]:
labels = {
    0: "خیر",
    1: "بله"
}

In [29]:
from tqdm import tqdm

def test_model(scenario, test_df, log_data, examples=None):
    correct = 0
    total = len(test_df)
    for index, row in tqdm(test_df.iterrows(), desc=f"Running {scenario} scenario..."):
        prompt = n_shot_prompt(scenario, row['q1'], row['q2'], examples)
        output = generated_text(prompt)
        label = int(row['label'])
        expected_output = labels[label]

        if expected_output in output:
            correct += 1

        log_data.append({'q1': row['q1'], 'q2': row['q2'], 'generated_text': output, 'real_label': labels[label], 'scenario': scenario})

    return correct / total

In [25]:
len(test_df)

1916

In [31]:
import pandas as pd
from tqdm import tqdm

# Assuming you have defined test_df and train_df somewhere in your code

log_data = []  # List to collect data for log_df

scenarios = ["Zero-shot", "One-shot", "Five-shot"]

test_subset_examples = test_df.sample(n=100, random_state=54)  # Subset of test data
one_shot_subset_examples = train_df.sample(n=5, random_state=54)  # Subset of training data for one-shot
grouped = train_df.groupby('label')
five_shot_subset_examples = grouped.apply(lambda x: x.sample(n=2 if x.name == '0' else 3, random_state=54))
five_shot_subset_examples = five_shot_subset_examples.reset_index(drop=True)  # Subset of training data for five-shot

# Define labels dictionary
labels = {
    0: "خیر",
    1: "بله"
}

def test_model(scenario, test_df, log_data, examples=None):
    correct = 0
    total = len(test_df)
    for index, row in tqdm(test_df.iterrows(), desc=f"Running {scenario} scenario..."):
        prompt = n_shot_prompt(scenario, row['q1'], row['q2'], examples)
        output = generated_text(prompt)
        label = int(row['label'])
        expected_output = labels[label]

        if expected_output in output:
            correct += 1

        #log_data.append({'q1': row['q1'], 'q2': row['q2'], 'generated_text': output, 'real_label': labels[label], 'scenario': scenario})

    return correct / total

zero_shot_result = None
one_shot_result = None
five_shot_result = None

for scenario in scenarios:
    if scenario == "Zero-shot":
        zero_shot_result = test_model(scenario, test_subset_examples, log_data, examples=None)
    elif scenario == "One-shot":
        one_shot_result = test_model(scenario, test_subset_examples, log_data, examples=one_shot_subset_examples)
    elif scenario == "Five-shot":
        five_shot_result = test_model(scenario, test_subset_examples, log_data, examples=five_shot_subset_examples)
    else:
        raise ValueError("Invalid scenario")

# Convert log_data to DataFrame
log_df = pd.DataFrame(log_data)

# Print results
print("Experiment result:")
print("Model Name: NousResearch/Meta-Llama-3-8B-Instruct")
print(f"Zero-shot accuracy: {zero_shot_result}")
print(f"One-shot accuracy: {one_shot_result}")
print(f"Five-shot accuracy: {five_shot_result}")


Running Zero-shot scenario...: 100it [01:58,  1.19s/it]
Running One-shot scenario...: 100it [02:38,  1.58s/it]
Running Five-shot scenario...: 100it [04:39,  2.79s/it]

Experiment result:
Model Name: NousResearch/Meta-Llama-3-8B-Instruct
Zero-shot accuracy: 0.65
One-shot accuracy: 0.48
Five-shot accuracy: 0.44
